In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping

In [2]:
df = pd.read_csv('Dataset/dataset_cleaned1.csv', low_memory=False)
df.columns

Index(['Maker', 'Model', 'Number_of_Cylinders', 'Engine_Type',
       'Engine_Horse_Power', 'Engine_Horse_Power_RPM', 'Transmission',
       'Mixed_Fuel_Consumption_per_100_km_l', 'Fuel_Tank_Capacity',
       'Acceleration_0_to_100_Km', 'Max_Speed_Km_per_Hour', 'Fuel_Grade',
       'Year', 'Type_of_Car'],
      dtype='object')

In [3]:
# Assuming 'Mixed_Fuel_Consumption_per_100_km_l' is the target variable
y = df['Mixed_Fuel_Consumption_per_100_km_l']

# Select specific columns as input features
selected_columns = ['Number_of_Cylinders',
                    'Engine_Type',
                    'Engine_Horse_Power',
                    'Engine_Horse_Power_RPM',
                    'Transmission',
                    'Acceleration_0_to_100_Km',
                    'Fuel_Grade']

X = df[selected_columns]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [4]:
# Build the TensorFlow Model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Assuming a regression problem
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])  # Add 'mae' as a metric
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, 
                    validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
372/372 [==============================] - 2s 4ms/step - loss: 9.6401 - mae: 2.1301 - val_loss: 2.8149 - val_mae: 1.2607
Epoch 2/50
372/372 [==============================] - 1s 3ms/step - loss: 2.3528 - mae: 1.1476 - val_loss: 1.9695 - val_mae: 1.0419
Epoch 3/50
372/372 [==============================] - 1s 3ms/step - loss: 1.7431 - mae: 0.9771 - val_loss: 1.6404 - val_mae: 0.9328
Epoch 4/50
372/372 [==============================] - 1s 3ms/step - loss: 1.5308 - mae: 0.9083 - val_loss: 1.5027 - val_mae: 0.8936
Epoch 5/50
372/372 [==============================] - 1s 3ms/step - loss: 1.4497 - mae: 0.8833 - val_loss: 1.4469 - val_mae: 0.8862
Epoch 6/50
372/372 [==============================] - 1s 3ms/step - loss: 1.4162 - mae: 0.8728 - val_loss: 1.4157 - val_mae: 0.8684
Epoch 7/50
372/372 [==============================] - 1s 3ms/step - loss: 1.3917 - mae: 0.8665 - val_loss: 1.3786 - val_mae: 0.8568
Epoch 8/50
372/372 [==============================] - 1s 3ms/step - loss: 1.

In [5]:
# Evaluate the model on the test set
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f'Mean Squared Error on Test Set: {loss}')
print(f'Mean Absolute Error on Test Set: {mae}')

117/117 [==============================] - 0s 2ms/step - loss: 1.2467 - mae: 0.8225
Mean Squared Error on Test Set: 1.2467138767242432
Mean Absolute Error on Test Set: 0.8225128650665283


In [6]:
import joblib

# Save the model to an HDF5 file
model.save('model_dinova1.h5')

# Save the scaler to a joblib file
joblib.dump(scaler, 'scaler_dinova.joblib')

['scaler_dinova.joblib']